In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

### look for duals in some sep bin

In [ ]:
df2 = pd.read_pickle("/home/insepien/research-data/pop-result/separation.pkl")
sep_kpc = df2['sep_kpc']

def find_agn_in_bin(bin_num):
    bins = np.logspace(-1,1.5,10)
    bin0_inds = np.where((bins[bin_num] <= sep_kpc) & (sep_kpc <= bins[bin_num+1]))
    bin0_agns = df2.loc[[i if i<14 else i+1 for i in bin0_inds[0]]]
    return bin0_agns

#find_agn_in_bin(2)
df2

### check J1010 calculations

In [ ]:
## J1010 CHECKS
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.io import fits
from astropy.cosmology import WMAP9 as cosmo
import astropy.units as u
data = fits.getdata('/home/insepien/research-data/alpaka/ALPAKA_v1_withDes.fits')
# get j1010 row
alpaka = Table(data).to_pandas()
j1010 = alpaka[alpaka['Desig']=="J1010+1413"]
lums = [i for i in list(j1010.keys()) if 'LUM' in i]

# OIII
ja_oiii = 1.2e44 # different from alpaka value??
i1 = ja_oiii*800/(1+1.3)
i2 = i1*1.3
print(f"alpaka OIII lum extinction corrected: {j1010['OIII_5007_LUM_DERRED'].values[0]:.1e}")
print(f"paper OIII lum: {ja_oiii:.1e}")
print(f"by paper lum, core 1 bolometric: {i1:.1e}, core 2 bolometric: {i2:.1e}")

# IR -- use Planck13 here
f160 = 3.23e-12*u.erg/u.s/(u.cm)**2 # from paper
ja_ir = 1.1e11
from astropy.cosmology import Planck13
da = Planck13.luminosity_distance(j1010['Z'].values[0])
print(f"paper IR lum: {ja_ir:.1e}")
print(f"IR Lum: {(f160*4*np.pi*da**2).to(u.L_sun):.1e}")

In [ ]:
# checking J1010 WISE numbers, but the paper number is actually L_AGN, so this might not be a fair comparison

import astropy.constants as const
wmags_ = np.array([13.056, 11.208, 6.784, 3.816])

zeromagflux = np.array([309.540, 171.787, 31.674, 8.363])*u.Jy
fluxdens = zeromagflux*10**(-wmags_/2.5)
## get flux at obs wavelength
obs_flux = fluxdens[-1]
## change to luminosity
obs_hz = (const.c/(22*u.micron)).to(u.Hz)
ja_ir = 6e46
mylum = (obs_flux*obs_hz*4*np.pi*Planck13.luminosity_distance(0.2)**2).to(u.erg/u.s).value
print(f"paper IR lum?: {ja_ir:.1e}")
print(f"IR Lum: {mylum:.1e}")

### calculate bolometric luminosity from WISE mIR and Hopkins+07 corrections to 15 nm

In [ ]:
from scipy.optimize import fsolve
import astropy.constants as const

def wise_lum_from_mag(wmags_, wmags_err_, rf_wavelength_, redshift_):
    """turn wise mags into wise flux at restframe 15 microns"""
    ## get observed wavelength corresponding to global rest-frame wavelength
    obs_wavelength = rf_wavelength_*(1+redshift_)
  
    ## change mags to fluxes -- http://wise2.ipac.caltech.edu/docs/release/allsky/expsup/sec4_4h.html#example
    zeromagflux = np.array([309.540, 171.787, 31.674, 8.363])*u.Jy
    fluxdens = zeromagflux*10**(-wmags_/2.5)
    fluxdens_err = zeromagflux*10**(-wmags_err_/2.5)
  
    ## interpolate - use straight line
    wise_wavelengths = np.array([3.4, 4.6, 12., 22.])
    wiseflux = np.polyfit(wise_wavelengths, fluxdens.value,1, w=1./fluxdens_err)
  
  
    ## get flux at obs wavelength, i.e. just a straight line here
    obs_flux = (wiseflux[0]*obs_wavelength+wiseflux[1])*u.Jy
  
    ## change to luminosity
    obs_hz = (const.c/(obs_wavelength*u.micron)).to(u.Hz)
    lum = (obs_flux*obs_hz*4*np.pi*
           cosmo.luminosity_distance(redshift_)**2).to(u.erg/u.s)
    return lum

# this file is from query of sources on WISE, instructions for query in kris candidate selection notebook
wise = pd.read_pickle("/home/insepien/research-data/alpaka/wise_39fits.pkl")
wise.fillna(0,inplace=True)
# read alpaka for optical comparison and redshift
alpaka = pd.read_pickle("/home/insepien/research-data/alpaka/alpaka_39fits.pkl")
alpaka.reset_index(inplace=True)

## get wise mags and errors
w1mag = wise['w1mpro']
w2mag = wise['w2mpro']
w3mag = wise['w3mpro']
w4mag = wise['w4mpro']
wmags = np.array([w1mag, w2mag, w3mag, w4mag])
wmags_err = np.array([wise['w1sigmpro'], wise['w2sigmpro'], wise['w3sigmpro'], wise['w4sigmpro']])

# calculate 15 microns lum
wise_lums = np.zeros((len(wise)))
for i in range(0, len(wise)):
    on = wise.loc[i,"designation"]
    z = np.mean(alpaka[alpaka['Desig']==on]['Z'])
    wise_lums[i] = wise_lum_from_mag(wmags[:,i], wmags_err[:,i], 15., alpaka.loc[i,"Z"]).value

# bolometric correction from Hopkins
with open("/home/insepien/research-data/pop-result/bc.txt","r") as f:
    d = f.read().splitlines()
hopkins = pd.DataFrame([d[1:][i].split(' ') for i in range(len(d[1:]))],columns=d[0].split(' '))
Lbol = np.array(list(hopkins['Lbols'].values), dtype=float)
LIR = np.array(list(hopkins['LIRs'].values), dtype=float)

# interpolate wise IR lum
from scipy.interpolate import CubicSpline
spl = CubicSpline(LIR, Lbol)
irbol = 10**(spl(np.log10(wise_lums)))

In [ ]:
# compare OIII and IR bolo lum
df_all = pd.read_pickle("/home/insepien/research-data/pop-result/all_results_updated.pkl")
oiiibol = np.array(list(df_all['OIII_BOL'].values))
plt.hist(np.log10(oiiibol),label="OIII Mullaney*800")
plt.hist(np.log10(irbol),label='wise IR corrected by Hopkins')
plt.xlabel("Log(L_bol)")
plt.ylabel("# quasars")
plt.title("Compare bolometric luminosity approximations from OIII and IR")
plt.legend();